In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [27]:
"""
from selenium import webdriver
import chromedriver_autoinstaller
import time

chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

driver = webdriver.Chrome()
driver.get("http://www.python.org")

time.sleep(10)
driver.quit()
"""

'\nfrom selenium import webdriver\nimport chromedriver_autoinstaller\nimport time\n\nchromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists\n                                      # and if it doesn\'t exist, download it automatically,\n                                      # then add chromedriver to path\n\ndriver = webdriver.Chrome()\ndriver.get("http://www.python.org")\n\ntime.sleep(10)\ndriver.quit()\n'

In [81]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Selenium için Chrome ayarları
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# WebDriver Manager ile otomatik ChromeDriver yönetimi
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Web sayfasını aç
url = "https://www.obilet.com/oteller/trabzon-277-91506_38283-2/20250801-20250807/2ad?so=price-highest"
driver.get(url)

print("Sayfa başarıyla yüklendi!")
driver.quit()


Sayfa başarıyla yüklendi!


In [82]:
%%time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Selenium için Chrome ayarları
chrome_options = Options()
chrome_options.add_argument("--headless")  # Tarayıcıyı gizli modda çalıştırır
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# WebDriver Manager ile otomatik ChromeDriver yönetimi
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Web sayfasını aç
url = "https://www.obilet.com/oteller/trabzon-277-91506_38283-2/20250801-20250807/2ad?so=price-highestt"
driver.get(url)

# Sonsuz kaydırma için ayar
otel_bilgileri = []
previous_height = driver.execute_script("return document.body.scrollHeight")
max_oteller = 20  # Çekmek istediğiniz maksimum otel sayısı

while len(otel_bilgileri) < max_oteller:
    # Sayfanın sonuna kaydır
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Yeni otellerin yüklenmesi için bekle

    # Sayfanın HTML içeriğini çek
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")

    # Şu ana kadar yüklenmiş tüm otelleri çek
    otel_listesi = soup.find_all("div", class_="hotel-item")
    
    # Her bir otelin bilgilerini çek
    for otel in otel_listesi:
        # Otel adı
        isim = otel.find("div", {"class": "hotel-item__name notranslate"})
        isim = isim.text.strip() if isim else "Bilinmiyor"

        # Otel adresi
        adres = otel.find("div", {"class": "hotel-location__address"})
        adres = adres.text.strip() if adres else "Bilinmiyor"

        # Şehir merkezine uzaklık
        merkeze_uzaklik = otel.find("div", {"class": "hotel-location__city-center-distance"})
        merkeze_uzaklik = merkeze_uzaklik.text.strip() if merkeze_uzaklik else "Bilinmiyor"

        # Yorum sayısı
        yorum_elementi = otel.find("span", {"class": "hotel-review__comment"})
        if yorum_elementi:
            yorum_sayisi = ''.join(filter(str.isdigit, yorum_elementi.text))  # Sadece rakamları al
        else:
            yorum_sayisi = "Bilinmiyor"

        # Puan
        puan = otel.find("span", {"class": "hotel-review__badge"})
        puan = puan.text.strip() if puan else "Bilinmiyor"

        # Yıldız sayısı
        otel_yildizi = otel.find("div", class_="hotel-item__star")
        if otel_yildizi:
            yildiz_sayisi = len(otel_yildizi.find_all("svg", class_="star"))
        else:
            yildiz_sayisi = "Bilinmiyor"

        # Fiyat
        fiyat = otel.find("div", {"class": "hotel-price__amount"})
        fiyat = fiyat.text.strip() if fiyat else "Bilinmiyor"

        # Ücretsiz İptal Bilgisi
        ucretsiz_iptal = otel.find("span")
        if ucretsiz_iptal and "Ücretsiz İptal" in ucretsiz_iptal.text:
            ucretsiz_iptal_bilgisi = ucretsiz_iptal.text.strip()
        else:
            ucretsiz_iptal_bilgisi = "Bilgi Yok"

        # Konaklama Tipi Bilgisi (Ultra Her Şey Dahil, vb.)
        konaklama_elementi = otel.find("div", {"class": "hotel-features__inner board-item"})
        if konaklama_elementi:
            konaklama_tipi_span = konaklama_elementi.find("span")
            konaklama_tipi = konaklama_tipi_span.text.strip() if konaklama_tipi_span else "Bilgi Yok"
        else:
            konaklama_tipi = "Bilgi Yok"

        # Otel bilgilerini bir sözlükte topla
        otel_bilgisi = {
            "İsim": isim,
            "Adres": adres,
            "Merkeze Uzaklık": merkeze_uzaklik,
            "Yorum Sayısı": yorum_sayisi,
            "Puan": puan,
            "Yıldız Sayısı": yildiz_sayisi,
            "Fiyat": fiyat,
            "Ücretsiz İptal": ucretsiz_iptal_bilgisi,
            "Konaklama Tipi": konaklama_tipi,
        }

        # Aynı otel bilgisini tekrar eklememek için kontrol et
        if otel_bilgisi not in otel_bilgileri:
            otel_bilgileri.append(otel_bilgisi)

    # Yeni yükleme olmadıysa döngüyü kır
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == previous_height:
        break  # Daha fazla otel yüklenmiyor
    previous_height = new_height

# Tarayıcıyı kapat
driver.quit()

# Verileri DataFrame'e çevir
df = pd.DataFrame(otel_bilgileri)

# DataFrame'i yazdır
print(df.head(2))  # İlk 2 kaydı göster


                İsim                 Adres Merkeze Uzaklık Yorum Sayısı Puan  \
0  Grand Vaves Hotel  Ortahisar, Trabzon -  Merkeze 2.6 km           49  9.3   
1        Zorlu Grand  Ortahisar, Trabzon -  Merkeze 0.2 km           30  9.0   

   Yıldız Sayısı       Fiyat  Ücretsiz İptal  Konaklama Tipi  
0              4  187.099 TL  Ücretsiz İptal  Yarım Pansiyon  
1              5   38.202 TL       Bilgi Yok       Bilgi Yok  
CPU times: total: 281 ms
Wall time: 15.2 s


In [83]:
df

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,Grand Vaves Hotel,"Ortahisar, Trabzon -",Merkeze 2.6 km,49,9.3,4,187.099 TL,Ücretsiz İptal,Yarım Pansiyon
1,Zorlu Grand,"Ortahisar, Trabzon -",Merkeze 0.2 km,30,9.0,5,38.202 TL,Bilgi Yok,Bilgi Yok
2,Melissa Suite Otel,"2 Nolu Bostancı, Ortahisar, Trabzon -",Merkeze 2.9 km,21,10,0,31.135 TL,Bilgi Yok,Bilgi Yok
3,Air Suite Hotel,"Ortahisar, Trabzon -",Merkeze 4.8 km,33,8.4,0,12.150 TL,Bilgi Yok,Bilgi Yok
4,Cephanelik Boutique Hotel,"Ortahisar, Trabzon -",Merkeze 1.4 km,25,7.9,3,29.686 TL,Ücretsiz İptal,Kahvaltı Dahil
5,Liva Teras,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,13,9.3,0,29.798 TL,Bilgi Yok,Bilgi Yok
6,Nova Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,84,8.0,4,27.863 TL,Ücretsiz İptal,Bilgi Yok
7,Ozkan Hotel,"Çaykara, Trabzon -",Bilinmiyor,22,8.4,0,26.387 TL,Bilgi Yok,Kahvaltı Dahil
8,My Home Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,18,7.5,4,48.704 TL,Ücretsiz İptal,Bilgi Yok
9,Sera Nature Hotel & Spa,"Akçaabat, Trabzon -",Bilinmiyor,63,9.0,4,30.653 TL,Ücretsiz İptal,Kahvaltı Dahil


In [84]:
df_filtered = df[(df['Puan'] != 10)]
df_filtered

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,Grand Vaves Hotel,"Ortahisar, Trabzon -",Merkeze 2.6 km,49,9.3,4,187.099 TL,Ücretsiz İptal,Yarım Pansiyon
1,Zorlu Grand,"Ortahisar, Trabzon -",Merkeze 0.2 km,30,9.0,5,38.202 TL,Bilgi Yok,Bilgi Yok
2,Melissa Suite Otel,"2 Nolu Bostancı, Ortahisar, Trabzon -",Merkeze 2.9 km,21,10,0,31.135 TL,Bilgi Yok,Bilgi Yok
3,Air Suite Hotel,"Ortahisar, Trabzon -",Merkeze 4.8 km,33,8.4,0,12.150 TL,Bilgi Yok,Bilgi Yok
4,Cephanelik Boutique Hotel,"Ortahisar, Trabzon -",Merkeze 1.4 km,25,7.9,3,29.686 TL,Ücretsiz İptal,Kahvaltı Dahil
5,Liva Teras,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,13,9.3,0,29.798 TL,Bilgi Yok,Bilgi Yok
6,Nova Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,84,8.0,4,27.863 TL,Ücretsiz İptal,Bilgi Yok
7,Ozkan Hotel,"Çaykara, Trabzon -",Bilinmiyor,22,8.4,0,26.387 TL,Bilgi Yok,Kahvaltı Dahil
8,My Home Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,18,7.5,4,48.704 TL,Ücretsiz İptal,Bilgi Yok
9,Sera Nature Hotel & Spa,"Akçaabat, Trabzon -",Bilinmiyor,63,9.0,4,30.653 TL,Ücretsiz İptal,Kahvaltı Dahil


In [85]:
df_filtered = df_filtered [(df_filtered['Puan'] != 'Bilinmiyor')]
df_filtered

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,Grand Vaves Hotel,"Ortahisar, Trabzon -",Merkeze 2.6 km,49,9.3,4,187.099 TL,Ücretsiz İptal,Yarım Pansiyon
1,Zorlu Grand,"Ortahisar, Trabzon -",Merkeze 0.2 km,30,9.0,5,38.202 TL,Bilgi Yok,Bilgi Yok
2,Melissa Suite Otel,"2 Nolu Bostancı, Ortahisar, Trabzon -",Merkeze 2.9 km,21,10,0,31.135 TL,Bilgi Yok,Bilgi Yok
3,Air Suite Hotel,"Ortahisar, Trabzon -",Merkeze 4.8 km,33,8.4,0,12.150 TL,Bilgi Yok,Bilgi Yok
4,Cephanelik Boutique Hotel,"Ortahisar, Trabzon -",Merkeze 1.4 km,25,7.9,3,29.686 TL,Ücretsiz İptal,Kahvaltı Dahil
5,Liva Teras,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,13,9.3,0,29.798 TL,Bilgi Yok,Bilgi Yok
6,Nova Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,84,8.0,4,27.863 TL,Ücretsiz İptal,Bilgi Yok
7,Ozkan Hotel,"Çaykara, Trabzon -",Bilinmiyor,22,8.4,0,26.387 TL,Bilgi Yok,Kahvaltı Dahil
8,My Home Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,18,7.5,4,48.704 TL,Ücretsiz İptal,Bilgi Yok
9,Sera Nature Hotel & Spa,"Akçaabat, Trabzon -",Bilinmiyor,63,9.0,4,30.653 TL,Ücretsiz İptal,Kahvaltı Dahil


In [86]:
df_filtered = df_filtered[(df_filtered['Fiyat'] != 'Bilinmiyor')]
df_filtered

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,Grand Vaves Hotel,"Ortahisar, Trabzon -",Merkeze 2.6 km,49,9.3,4,187.099 TL,Ücretsiz İptal,Yarım Pansiyon
1,Zorlu Grand,"Ortahisar, Trabzon -",Merkeze 0.2 km,30,9.0,5,38.202 TL,Bilgi Yok,Bilgi Yok
2,Melissa Suite Otel,"2 Nolu Bostancı, Ortahisar, Trabzon -",Merkeze 2.9 km,21,10,0,31.135 TL,Bilgi Yok,Bilgi Yok
3,Air Suite Hotel,"Ortahisar, Trabzon -",Merkeze 4.8 km,33,8.4,0,12.150 TL,Bilgi Yok,Bilgi Yok
4,Cephanelik Boutique Hotel,"Ortahisar, Trabzon -",Merkeze 1.4 km,25,7.9,3,29.686 TL,Ücretsiz İptal,Kahvaltı Dahil
5,Liva Teras,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,13,9.3,0,29.798 TL,Bilgi Yok,Bilgi Yok
6,Nova Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,84,8.0,4,27.863 TL,Ücretsiz İptal,Bilgi Yok
7,Ozkan Hotel,"Çaykara, Trabzon -",Bilinmiyor,22,8.4,0,26.387 TL,Bilgi Yok,Kahvaltı Dahil
8,My Home Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,18,7.5,4,48.704 TL,Ücretsiz İptal,Bilgi Yok
9,Sera Nature Hotel & Spa,"Akçaabat, Trabzon -",Bilinmiyor,63,9.0,4,30.653 TL,Ücretsiz İptal,Kahvaltı Dahil


In [87]:
df_reset = df_filtered.reset_index()
df_reset

,index,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,0,Grand Vaves Hotel,"Ortahisar, Trabzon -",Merkeze 2.6 km,49,9.3,4,187.099 TL,Ücretsiz İptal,Yarım Pansiyon
1,1,Zorlu Grand,"Ortahisar, Trabzon -",Merkeze 0.2 km,30,9.0,5,38.202 TL,Bilgi Yok,Bilgi Yok
2,2,Melissa Suite Otel,"2 Nolu Bostancı, Ortahisar, Trabzon -",Merkeze 2.9 km,21,10,0,31.135 TL,Bilgi Yok,Bilgi Yok
3,3,Air Suite Hotel,"Ortahisar, Trabzon -",Merkeze 4.8 km,33,8.4,0,12.150 TL,Bilgi Yok,Bilgi Yok
4,4,Cephanelik Boutique Hotel,"Ortahisar, Trabzon -",Merkeze 1.4 km,25,7.9,3,29.686 TL,Ücretsiz İptal,Kahvaltı Dahil
5,5,Liva Teras,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,13,9.3,0,29.798 TL,Bilgi Yok,Bilgi Yok
6,6,Nova Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,84,8.0,4,27.863 TL,Ücretsiz İptal,Bilgi Yok
7,7,Ozkan Hotel,"Çaykara, Trabzon -",Bilinmiyor,22,8.4,0,26.387 TL,Bilgi Yok,Kahvaltı Dahil
8,8,My Home Uzungöl,"Uzungöl, Çaykara, Trabzon -",Bilinmiyor,18,7.5,4,48.704 TL,Ücretsiz İptal,Bilgi Yok
9,9,Sera Nature Hotel & Spa,"Akçaabat, Trabzon -",Bilinmiyor,63,9.0,4,30.653 TL,Ücretsiz İptal,Kahvaltı Dahil


In [71]:
df_reset = df_reset.drop(df_reset.index[30:197])
df_reset

,index,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,2,Mandarin Oriental Bodrum,"Göltürkbükü, Bodrum, Muğla -",Bilinmiyor,326,9.4,5,532.173 TL,Bilgi Yok,Kahvaltı Dahil
1,3,Liberty Lykia,"Ölüdeniz, Fethiye, Muğla -",Bilinmiyor,30,8.9,5,408.752 TL,Bilgi Yok,Her Şey Dahil
2,4,D Maris Bay,"Hisarönü, Marmaris, Muğla -",Bilinmiyor,14,9.4,5,366.843 TL,Bilgi Yok,Kahvaltı Dahil
3,5,"Caresse, a Luxury Collection Resort & Spa, Bodrum","Bitez, Bodrum, Muğla -",Merkeze 3.9 km,16,9.4,5,346.853 TL,Bilgi Yok,Kahvaltı Dahil
4,6,Lujo Hotel Bodrum,"Bodrum, Muğla -",Bilinmiyor,29,9.1,5,344.400 TL,Bilgi Yok,Kahvaltı Dahil
5,7,Vogue Hotel Supreme Bodrum,"Torba, Bodrum, Muğla -",Merkeze 6.5 km,81,8.2,5,338.424 TL,Bilgi Yok,Ultra Her Şey Dahil
6,9,Titanic Luxury Collection Bodrum,"Bodrum, Muğla -",Bilinmiyor,55,9.2,5,278.190 TL,Bilgi Yok,Her Şey Dahil
7,10,Cragus Inn Fethiye,"Fethiye, Muğla -",Bilinmiyor,23,8.2,0,252.923 TL,Bilgi Yok,Bilgi Yok
8,11,Yalıkavak Marina Beach Hotel,"Yalıkavak, Bodrum, Muğla -",Bilinmiyor,36,8.6,5,243.285 TL,Bilgi Yok,Kahvaltı Dahil
9,12,Allium Bodrum Resort & Spa,"Bodrum, Muğla -",Bilinmiyor,19,9.4,3,241.497 TL,Bilgi Yok,Kahvaltı Dahil


In [88]:
df_reset.to_csv('iyi_trabzon.csv', index=False)